In [1]:
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
train_df = pd.read_csv('train.csv')
train_df['label'].unique()

test_df = pd.read_csv('test.csv')

In [3]:
train_df['label'] = train_df['label'].map({'building':0, 'cat':1, 'airplane':2, 'emotion_face':3,
                'rabbit':4, 'bird':5, 'truck':6, 'ball':7, 'police_car':8, 'apple':9})

In [4]:
train_X = train_df.iloc[:, 2:]
train_y = train_df['label']

In [5]:
scaler = StandardScaler()  # 또는 MinMaxScaler()
X_scaled = scaler.fit_transform(train_X)

In [6]:
pca = PCA(n_components=100)  # 예: 1024 → 100차원 축소
X_pca = pca.fit_transform(X_scaled)

In [7]:
X_test = test_df.iloc[:, 1:]   # 테스트 입력 벡터

In [8]:
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}

grid = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='accuracy')
grid.fit(train_X, train_y)

print("최적 모델:", grid.best_estimator_)
print("최고 정확도:", grid.best_score_)

최적 모델: RandomForestClassifier(max_depth=20)
최고 정확도: 0.840064510652746
